<a href="https://colab.research.google.com/github/AOSPITIA/PruebaProteccion_AlejandroOspitia/blob/main/PruebaTecnica_AlejandroOspitia_P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Prueba técnica - Punto 3**
**ESPECIALISTA DE CIENCIA DE DATOS**

**Alejandro Ospitia**

*Enunciado tercer punto*

En los últimos años, los modelos basados en mecanismos de autoatención han demostrado un desempeño sobresaliente en tareas de lenguaje natural. La empresa está evaluando si este tipo de modelos justifica su adopción frente a enfoques neuronales más tradicionales. Utilizando el mismo conjunto de reseñas del punto anterior, se solicita implementar un modelo Transformer preentrenado, realizando un ajuste adecuado para la tarea de clasificación de sentimiento. El objetivo no es únicamente maximizar métricas, sino analizar cómo este enfoque se comporta en comparación con la solución desarrollada previamente.

In [1]:
#Importación de librerias
import pandas as pd
import re
from bs4 import BeautifulSoup


import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from transformers import (DistilBertTokenizerFast, #Se llama desde HuggingFace
                          DistilBertForSequenceClassification,
                          Trainer,
                          TrainingArguments)

from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

Como la información que se implementará en el puntor 3 es la misma que se uso en el punto 2, se incluiran las transformaciones definidas en este punto.

In [4]:
#Carga de información y aplicaciones definidas para el dataframe
df = pd.read_table("/content/punto_2_3.csv")
df["sentiment"] = df["review,sentiment"].astype(str).str[-1].astype(int)
df["review"] = df["review,sentiment"].astype(str).str[:-2]
df = df[["review","sentiment"]]

def limpieza(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = text.lower()
    return text
df["review"] = df["review"].apply(limpieza)

In [5]:
#Conformación de dataframes teniendo en cuenta la velocidad del procesamiento
#Procesamiento completo
df_original = df.copy()
#Procesamiento parcial
df_prueba = df.copy()
df_prueba = df_prueba.sample(5000)

df = df_prueba.copy()
# df = df_original.copy()

#**Desarrollo del módelo**

Creación del modelo
* Creación de features para el modelo
* Tokenización de features a través del modelo escogido
* División de la base para entrenamiento (70), validación (10) y testeo (20)
* Creación vocabulario
* Conversión texto a secuencias
* Medición y validación

In [6]:
X = df["review"].values
y = df["sentiment"].values

In [7]:
# Generación 70% train, 30% temporal (10% Validación, 20% testeo)
X_train, X_temp, y_train, y_temp = train_test_split(X, y,test_size=0.3,random_state=42,stratify=y)
# Dividir 30% temp en 10% validación y 20% testeo
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp,test_size=2/3,random_state=42,stratify=y_temp)

print("Train:", len(X_train)/len(X))
print("Validation:", len(X_val)/len(X))
print("Test:", len(X_test)/len(X))

Train: 0.7
Validation: 0.1
Test: 0.2


In [8]:
#Tokenización a través del modelo
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

MAX_LENGTH = 256
def tokenizar(texts):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH)

train_encodings = tokenizar(X_train)
val_encodings   = tokenizar(X_val)
test_encodings  = tokenizar(X_test)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
#Implementación dataset con Pytorch
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, y_train)
val_dataset   = SentimentDataset(val_encodings, y_val)
test_dataset  = SentimentDataset(test_encodings, y_test)

**Selección modelo**

Se selecciona el modelo distilbert donde se usa el caso uncased (sin mayúsculas aprovechando el tratamiento que se hizo en el punto 2). La elección de este modelo se basa en lo siguiente:
* Es más liviano que BERT
* Tiene buen desempeño
* Entrenamiento rápido (se debe tener en cuenta la capacidad de la máquina)

In [10]:
#Generación del modelo preentrenado
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
#Generación de métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {"accuracy": accuracy_score(labels, preds),
            "precision": precision_score(labels, preds),
            "recall": recall_score(labels, preds),
            "f1": f1_score(labels, preds)}

In [13]:
import transformers
print(transformers.__version__)

4.57.6


In [ ]:
#Entrenamiento del modelo
#Configuración de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1")

#Aplicación entrenamiento
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


In [ ]:
#Evaluación del modelo
test_results = trainer.evaluate(test_dataset)
print(test_results)

In [ ]:
#Comparativa
pred_test = trainer.predict(test_dataset)
y_pred = pred_test.predictions.argmax(axis=1)

In [ ]:
#Reporte de clasificación
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
#Generación de gráficas de las métricas del modelo
log_history = pd.DataFrame(trainer.state.log_history)
#Accuracy (Precisión)
plt.figure()
plt.plot(log_history["eval_accuracy"].dropna(), marker="o", label="Validación Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("validación Accuracy")
plt.legend()
plt.show()

#Loss (perdida)
plt.figure()
plt.plot(log_history["loss"].dropna(), marker="o", label="Entramiento Loss")
plt.plot(log_history["eval_loss"].dropna(), marker="o", label="Validación Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Entrenamiento vs validación Loss")
plt.legend()
plt.show()

#**Conclusiones**

1. Al aplicar un modelo preentrenado se disminuye el tiempo en la estructuración del modelo partiendo de cero.
2. Se seleccionó el modelo distilbert uncased aprovechando el tratamiento previo, por lo que ss más liviano que BERT, tiene buen desempeño y y un entrenamiento rápido.
3. No se alcanzó a ejecutar de manera completa el código del modelo con el modelo Preentrenado, razón por la cual no se efectuó la comparación.